# 准备数据

In [5]:
import sys

sys.path.append('../')

from alignment_utils import one_file_per_process
from alignment_utils import extract_and_combine_numbers
from alignment_utils import extract_and_combine_numbers_in_not_start
from alignment_utils import longest_increasing_subsequence_index
from alignment_utils import find_answer_split_str
from alignment_utils import find_next_question_index
from alignment_utils import refine_answers
from alignment_utils import match_specific_from_end
from alignment_utils import answer_area_str_process
from alignment_utils import generate_answer_area_string
from alignment_utils import align_answers_in_questions
from alignment_utils import match_specific_from_start
from alignment_utils import type_of_judgment
from alignment_utils import split_question
from alignment_utils import find_continuous_sequence

In [6]:
import os
import glob
from pathlib import Path

examination_paper_list = []

path = Path("./answer_markdown")

for file in path.glob("*.md"):
    with open(file, "r", encoding="utf-8") as f:
        examination_paper_list.append({
            "file_path": str(file),
            "text": one_file_per_process(f.read())
        })
len(examination_paper_list)

1615

# 寻找题目

In [45]:
def get_all_question(text):
    # 拆分成行
    lines = text.splitlines()
    
    # 定义不准确的题目列表    
    inaccuracy_question = []

    # 从0的位置寻找第一道题
    index = find_next_question_index(0, lines)
    
    while index < len(lines):
        # 寻找下一个题目的index
        next_index = find_next_question_index(index, lines)
        
        inaccuracy_question.append("\n".join(lines[index: next_index]))
        index = next_index
        
#     print([(extract_and_combine_numbers(topic), i) for i, topic in enumerate(inaccuracy_question) if extract_and_combine_numbers(topic) is not None])
    # 通过"最长递增子序列"寻找每个精准的题目所在inaccuracy_question对应的下标
    all_question_indexs = longest_increasing_subsequence_index(inaccuracy_question)
    

    # 定义准确的题目列表    
    all_question = []
    # index为all_question_indexs的下标，all_question_indexs[index]为inaccuracy_question的下标
    for index, question_index in enumerate(all_question_indexs):
        if index == len(all_question_indexs) -1 :
            all_question.append("\n".join(inaccuracy_question[question_index:]))
            break
            
        all_question.append("\n".join(inaccuracy_question[question_index:all_question_indexs[index+1]]))
#     for i in all_question:
#         print(i)
#         print("==========")
    if not all_question:
        return None, None
    
    all_question = find_continuous_sequence(all_question)
    
    print(all_question[-1])
    if text.splitlines()[0] in all_question[-1]:
        answer_split_str = text.splitlines()[0]
        # 看看试卷的title是否出现在"all_question[-1]"位置，如果出现则删除
        all_question[-1] = all_question[-1].split(text.splitlines()[0],1)[0]
    else:
        # 尝试寻找用于分割答题区与答案区的字符串，返回值为int/str，如果是str则是分割的字符串
        # 本质是在"all_question[-1]"寻找答案关键字等字样
        answer_split_str = find_answer_split_str(all_question)

        if isinstance(answer_split_str, str):
            # 如果找到这个拆分的字符串了，则先把最后一道题的内容进行拆分
            all_question[-1] = all_question[-1].split(answer_split_str)[0]
        
    return all_question, answer_split_str

# 创建测试样本

In [8]:
#1087 一 1.2.3 二 1.2.3 （规则还没写）
#1482 解析错误 暂未解决（规则BUG）

In [46]:
import random
random_number = random.randint(0, len(examination_paper_list))
# print(random_number)
# test_text = examination_paper_list[random_number]["text"]
test_text = examination_paper_list[100]["text"]

In [47]:
file_path = "answer_markdown\\（精校版）2020年全国卷Ⅰ理综化学高考试题文档版（含答案）.md"
test_text = list(filter(lambda row:file_path in row["file_path"], examination_paper_list))[0]["text"]

In [48]:
test_text.splitlines()

['![](./notebook/image/media/image3.png)绝密★启用前',
 '2020年普通高等学校招生全国统一考试',
 '理科综合能力测试 化学',
 '注意事项：',
 '1．答卷前，考生务必将自己的姓名、准考证号填写在答题卡上。',
 '2．回答选择题时，选出每小题答案后，用铅笔把答题卡上对应题目的答案标号涂黑，如需改动，用橡皮擦干净后，再选涂其它答案标号。回答非选择题时，将答案写在答题卡上，写在本试卷上无效。',
 '3．考试结束后，将本试卷和答题卡一并交回。',
 '可能用到的相对原子质量：H 1 C 12 N 14 O 16 Na 23 Al 27 P 31 S 32 Cl 35.5 V 51 Fe 56',
 '一、选择题：本题共13个小题，每小题6分。共78分，在每小题给出的四个选项中，只有一项是符合题目要求的。',
 '7．国家卫健委公布的新型冠状病毒肺炎诊疗方案指出，乙醚、75%乙醇、含氯消毒剂、过氧乙酸(CH~3~COOOH)、氯仿等均可有效灭活病毒。对于上述化学药品，下列说法错误的是',
 'A．CH~3~CH~2~OH能与水互溶',
 'B．NaClO通过氧化灭活病毒',
 'C．过氧乙酸相对分子质量为76',
 'D．氯仿的化学名称是四氯化碳',
 '8．紫花前胡醇![](./notebook/image/media/image4.png)可从中药材当归和白芷中提取得到，能提高人体免疫力。有关该化合物，下列叙述错误的是',
 'A．分子式为C~14~H~14~O~4~',
 'B．不能使酸性重铬酸钾溶液变色',
 'C．能够发生水解反应',
 'D．能够发生消去反应生成双键',
 '9．下列气体去除杂质的方法中，不能实现目的的是',
 '  ----- ---------------- ----------------------',
 '        气体（杂质）     方法',
 '  A．   SO~2~（H~2~S）   通过酸性高锰酸钾溶液',
 '  B．   Cl~2~（HCl）     通过饱和的食盐水',
 '  C．   N~2~（O~2~）     通过灼热的铜丝网',
 '  D．   NO（NO~2~）      通过氢氧化钠溶液',
 '  ----- --------------

In [49]:
# split_str用于进行下一步的分析（有可能这份试卷是有答题区和答案区，也有可能每道题的答案都在题的下方，还有可能根本没有答案）
all_question, split_str = get_all_question(test_text)
print(f"split_str:\n{split_str}")

if split_str in [-1, 0]:
    print("此试卷无答案")
    
all_question

36．\[化学------选修5：有机化学基础\]（15分）
有机碱，例如二甲基胺（![](./notebook/image/media/image34.png)）、苯胺（![](./notebook/image/media/image35.png)），吡啶（![](./notebook/image/media/image36.png)）等，在有机合成中应用很普遍，目前"有机超强碱"的研究越来越受到关注，以下为有机超强碱F的合成路线：
![](./notebook/image/media/image37.png)
已知如下信息：
①H~2~C=CH~2~![](./notebook/image/media/image39.png)
②![](./notebook/image/media/image40.png)+RNH~2~![](./notebook/image/media/image42.png)
③苯胺与甲基吡啶互为芳香同分异构体
回答下列问题：
（1）A的化学名称为\_\_\_\_\_\_\_\_。
（2）由B生成C的化学方程式为\_\_\_\_\_\_\_\_。
（3）C中所含官能团的名称为\_\_\_\_\_\_\_\_。
（4）由C生成D的反应类型为\_\_\_\_\_\_\_\_。
（5）D的结构简式为\_\_\_\_\_\_\_\_。
（6）E的六元环芳香同分异构体中，能与金属钠反应，且核磁共振氢谱有四组峰，峰面积之比为6∶2∶2∶1的有\_\_\_\_\_\_\_\_种，其中，芳香环上为二取代的结构简式为\_\_\_\_\_\_\_\_。
2020年普通高等学校招生全国统一考试
理科综合 化学参考答案
7．D 8．B 9．A 10．C 11．B 12．D 13．C
26．（1）加快酸浸和氧化反应速率（促进氧化完全）
（2）Fe^2+^ VO^+^+MnO~2~ +2H^+^ =+Mn^2+^+H~2~O
（3）Mn^2+^ Al^3+^和Fe^3+^
（4）Fe(OH)~3~
（5）NaAl(OH)~4~+ HCl= Al(OH)~3~↓+NaCl+H~2~O
（6）利用同离子效应，促进NH~4~VO~3~尽可能析出完全
27．（1）烧杯、量筒、托盘天平
（2）KCl
（3）石墨
（4）0.09 mol·L^−1^
（5）Fe^3+^+e^−^=

['1．答卷前，考生务必将自己的姓名、准考证号填写在答题卡上。',
 '2．回答选择题时，选出每小题答案后，用铅笔把答题卡上对应题目的答案标号涂黑，如需改动，用橡皮擦干净后，再选涂其它答案标号。回答非选择题时，将答案写在答题卡上，写在本试卷上无效。',
 '3．考试结束后，将本试卷和答题卡一并交回。\n可能用到的相对原子质量：H 1 C 12 N 14 O 16 Na 23 Al 27 P 31 S 32 Cl 35.5 V 51 Fe 56\n一、选择题：本题共13个小题，每小题6分。共78分，在每小题给出的四个选项中，只有一项是符合题目要求的。',
 '7．国家卫健委公布的新型冠状病毒肺炎诊疗方案指出，乙醚、75%乙醇、含氯消毒剂、过氧乙酸(CH~3~COOOH)、氯仿等均可有效灭活病毒。对于上述化学药品，下列说法错误的是\nA．CH~3~CH~2~OH能与水互溶\nB．NaClO通过氧化灭活病毒\nC．过氧乙酸相对分子质量为76\nD．氯仿的化学名称是四氯化碳',
 '8．紫花前胡醇![](./notebook/image/media/image4.png)可从中药材当归和白芷中提取得到，能提高人体免疫力。有关该化合物，下列叙述错误的是\nA．分子式为C~14~H~14~O~4~\nB．不能使酸性重铬酸钾溶液变色\nC．能够发生水解反应\nD．能够发生消去反应生成双键',
 '9．下列气体去除杂质的方法中，不能实现目的的是\n  ----- ---------------- ----------------------\n        气体（杂质）     方法\n  A．   SO~2~（H~2~S）   通过酸性高锰酸钾溶液\n  B．   Cl~2~（HCl）     通过饱和的食盐水\n  C．   N~2~（O~2~）     通过灼热的铜丝网\n  D．   NO（NO~2~）      通过氢氧化钠溶液\n  ----- ---------------- ----------------------',
 '10．铑的配合物离子\\[Rh(CO)~2~I~2~\\]^－^可催化甲醇羰基化，反应过程如图所示。\n![](./notebook/image/media/image5.png)\n下列叙述错误的是\nA．CH~3~COI是反应中间体\n

In [50]:
if split_str == 1:
    answers_with_questions = align_answers_in_questions(all_question)
#     print(check_sequence_is_full(answers_with_questions))
    for row in align_answers_in_questions(all_question):
        print()
        print(row["question"][:3])
        print("-------------------------------------------------------")
        print(f"{row['answer']}")
        print()
        print("============================================================")


8．紫
-------------------------------------------------------
C．能够发生水解反应
D．能够发生消去反应生成双键


13．
-------------------------------------------------------
三、非选择题：共174分，第22\~32题为必考题，每个试题考生都必须作答。第33\~38题为选考题，考生根据要求作答。
（一）必考题：共129分。


26．
-------------------------------------------------------
回答下列问题：
（1）"酸浸氧化"需要加热，其原因是\_\_\_\_\_\_\_\_\_\_\_。
（2）"酸浸氧化"中，VO^+^和VO^2+^被氧化成![](./notebook/image/media/image21.wmf)，同时还有\_\_\_\_\_\_\_\_\_\_\_离子被氧化。写出VO^+^转化为![](./notebook/image/media/image21.wmf)反应的离子方程式\_\_\_\_\_\_\_\_\_\_\_。
（3）"中和沉淀"中，钒水解并沉淀为![](./notebook/image/media/image22.wmf)，随滤液②可除去金属离子K^+^、Mg^2+^、Na^+^、\_\_\_\_\_\_\_\_\_\_\_，以及部分的\_\_\_\_\_\_\_\_\_\_\_。
（4）"沉淀转溶"中，![](./notebook/image/media/image22.wmf)转化为钒酸盐溶解。滤渣③的主要成分是\_\_\_\_\_\_\_\_\_\_\_。
（5）"调pH"中有沉淀生产，生成沉淀反应的化学方程式是\_\_\_\_\_\_\_\_\_\_\_。
（6）"沉钒"中析出NH~4~VO~3~晶体时，需要加入过量NH~4~Cl，其原因是\_\_\_\_\_\_\_\_\_\_\_。


27．
-------------------------------------------------------
回答下列问题：
（1）由FeSO~4~·7H~2~O固体配制0.10 mol·L^−1^ FeSO~4~溶液，需要的仪器有药匙、玻璃棒、\_\_\

# 寻找答案（试卷存在答题区与答案区）

In [19]:
# import re

# def get_all_answer_sequence(answer_area_string):
#     lines = answer_area_string.splitlines()
    
#     pattern = r"(\d+\\*[．. ]|\\*[（(]\d+\\*[）)]|^\\*\d+\\\.|^\d+、|\d+[ABCD])"
#     inaccuracy_answers = re.split(pattern, answer_area_string)
    
#     inaccuracy_answer_indexes = longest_increasing_subsequence_index(inaccuracy_answers)
    
#     processed_inaccuracy_answers = []
#     for index, answer_index in enumerate(inaccuracy_answer_indexes):
#         if index == len(inaccuracy_answer_indexes) -1 :
#             processed_inaccuracy_answers.append(inaccuracy_answers[answer_index])
#             break
            
#         processed_inaccuracy_answers.append("".join(inaccuracy_answers[answer_index:inaccuracy_answer_indexes[index+1]]))
        
#     return processed_inaccuracy_answers
def get_all_answer_sequence(answer_area_string):
    lines = answer_area_string.splitlines()
    
    inaccuracy_answers = []

    index = find_next_question_index(0, lines)
    while index < len(lines):
        next_index = find_next_question_index(index, lines)
       
        inaccuracy_answers.append("\n".join(lines[index: next_index]))
        index = next_index
        
#     print("============")
#     for i in inaccuracy_answers:
#         print(i)
#         print()
        
#     print([extract_and_combine_numbers(topic) for topic in inaccuracy_answers])
    inaccuracy_answer_indexes = longest_increasing_subsequence_index(inaccuracy_answers)
   
    processed_inaccuracy_answers = []
    for index, answer_index in enumerate(inaccuracy_answer_indexes):
        if index == len(inaccuracy_answer_indexes) -1 :
            processed_inaccuracy_answers.append(inaccuracy_answers[answer_index])
            break
        processed_inaccuracy_answers.append("\n".join(inaccuracy_answers[answer_index:inaccuracy_answer_indexes[index+1]]))
        
    return refine_answers(processed_inaccuracy_answers)[::-1]

In [20]:
answer_str = generate_answer_area_string(test_text, split_str)
answer_str

TypeError: must be str or None, not int

In [21]:
answer_str.splitlines()

NameError: name 'answer_str' is not defined

In [22]:
for i in get_all_answer_sequence(answer_area_str_process(answer_str)):
    print(i)
    print("=================")

NameError: name 'answer_str' is not defined

# 对齐

In [28]:
def alignment_answer(all_question, answer_str):
    questions_with_answer = []
    if not answer_str:
        return [{"question":question, "answer":None}for question in all_question]
    all_answer = get_all_answer_sequence(answer_area_str_process(answer_str))
    
    #  倒叙是因为，有的文件出现多个相同的题号，我们确保我们拿到的是准确的
    questions_map = {extract_and_combine_numbers_in_not_start(question):question for question in reversed(all_question)} 
    answer_map = {extract_and_combine_numbers_in_not_start(answer):answer for answer in reversed(all_answer)}

    for sequence_number in questions_map:
        questions_with_answer.append({
            "question":questions_map.get(sequence_number),
            "answer":answer_map.get(sequence_number, None)
        })
        
    return list(reversed(questions_with_answer))

In [32]:
def check_sequence_is_full(questions_with_answer):
    for index, question_map in enumerate(questions_with_answer):
        if index == len(questions_with_answer)-1:
            break
            
        current_number = extract_and_combine_numbers_in_not_start(question_map["question"])
        next_number = extract_and_combine_numbers_in_not_start(questions_with_answer[index+1]["question"])
        
        if not current_number or not next_number:
            return False
        
        if current_number + 1 != next_number:
            return False
      
    return True

In [25]:
for row in alignment_answer(all_question, answer_str):
    print()
#     print(row["question"][:2])
    print(row["question"])
    print("-------------------------------------------------------")
    print(f"{row['answer']}")
    print()
    print("============================================================")

NameError: name 'answer_str' is not defined

# 生产测试

In [33]:
import csv
from tqdm import tqdm

csv_path = "./questions_with_answers.csv"
loss_answer_file = set()
loss_question_file = set()
error_file = set()
check_erroe_file = set()

with open(csv_path, 'a', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['file_path', 'question', 'answer', 'is_full', 'is_error']  # 列名
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for examination_paper in tqdm(examination_paper_list):
#         try:
            file_path = examination_paper["file_path"]
            text = examination_paper["text"]

            all_question, split_str = get_all_question(text)

            # 判断试卷你是否有答案 -1 识别不到 0 无答案 
            if split_str in [-1, 0]: 
                loss_answer_file.add(file_path)
                continue

            # 无法找到题目
            if not all_question:
                loss_question_file.add(file_path)
                continue

            # 答案在每道题的下方split_str == 1，如果答案在卷子下方与题目隔离则split_str是返回隔离答案和题目那一行的str字符串
            if split_str == 1:
                questions_with_answers = align_answers_in_questions(all_question)
            else:
                answer_str = generate_answer_area_string(text, split_str)
                questions_with_answers = alignment_answer(all_question, answer_str)

            #  如果题目与答案都无法匹配
            if questions_with_answers is []:
                error_file.add(file_path)
                continue
            
            is_full = check_sequence_is_full(questions_with_answers)
           
            if not is_full:
                check_erroe_file.add(file_path)

            for question_with_answer in questions_with_answers:

                if question_with_answer is None:
                    question_with_answer = {"question":None,"answer":None}

                question_with_answer.update({"is_full":is_full,"file_path":file_path,"is_error":False})

                if question_with_answer is None:
                    question_with_answer.update({"is_error":True})

                if len(question_with_answer) == 5:
                        writer.writerow(question_with_answer)
#                 else:
#                     print(question_with_answer)
#         except:
#             print(file_path)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1615/1615 [00:03<00:00, 516.60it/s]


In [34]:
len(loss_answer_file | loss_question_file | error_file | check_erroe_file) / len(examination_paper_list)

0.31021671826625385

In [54]:
len(check_erroe_file)

244

In [36]:
check_erroe_file

{'answer_markdown\\2003数学二.md',
 'answer_markdown\\2004年上海市普通高等学校春季招生考.md',
 'answer_markdown\\2006年普通高等学校招生全国统一考试（四川卷.文）答案.md',
 'answer_markdown\\2006年普通高等学校招生全国统一考试（陕西卷.文）含答案.md',
 'answer_markdown\\2006年高考数学试题全国II卷文科.md',
 'answer_markdown\\2007年普通高等学校招生全国统一考试(安徽卷)数学文科详细解析（包括选择题填空题）.md',
 'answer_markdown\\2007年普通高等学校招生全国统一考试(安徽卷)数学理科详细解析（包括选择题填空题）.md',
 'answer_markdown\\2007年普通高等学校招生全国统一考试上海卷（数学文）.md',
 'answer_markdown\\2007年普通高等学校招生全国统一考试辽宁卷（数学文）.md',
 'answer_markdown\\2007年普通高等学校招生全国统一考试（全国卷II）文综试卷参考答案.md',
 'answer_markdown\\2007年普通高等学校招生全国统一考试（全国卷I）文综试卷参考答案.md',
 'answer_markdown\\2007年普通高等学校招生全国统一考试（四川卷）文综试卷参考答案.md',
 'answer_markdown\\2007年普通高等学校招生全国统一考试（宁夏卷）文综试卷参考答案.md',
 'answer_markdown\\2007年普通高等学校招生全国统一考试（福建卷）数学（文科）试卷参考答案.md',
 'answer_markdown\\2007年普通高等学校招生全国统一考试（重庆卷）文综试卷参考答案.md',
 'answer_markdown\\2007数学二.md',
 'answer_markdown\\2008年上海市春季卷.md',
 'answer_markdown\\2008年全国卷1.md',
 'answer_markdown\\2008年全国高等学校统一招生考试（江苏省）.md',
 'answer_markdown\\2008年北京卷.md',
 'ans

# 检验

In [37]:
import pandas as pd

df = pd.read_csv(csv_path)

# 计算每个file_path的总行数和非空answer的行数
file_path_counts = df['file_path'].value_counts()
file_path_answer_counts = df[df['answer'].notnull()]['file_path'].value_counts()

# 对齐这两个Series
file_path_counts, file_path_answer_counts = file_path_counts.align(file_path_answer_counts, fill_value=0)

# 找出那些所有相关行的answer都不为空的file_path
file_paths_with_all_answers = file_path_counts[file_path_counts == file_path_answer_counts]

# 计算这些file_path的数量
count_file_path_with_all_answers = len(file_paths_with_all_answers)

# 统计所有file_path的数量
count_all_file_path = df['file_path'].nunique()

count_file_path_with_all_answers / count_all_file_path

0.6710914454277286

In [38]:
df[df['answer'].isnull()]['file_path'].unique()

array(['answer_markdown\\2003年春季高考（北京）数学（理）试题.md',
       'answer_markdown\\2003数学（文）.md', 'answer_markdown\\2004-7数学（文）.md',
       'answer_markdown\\2004-数学（文）3.md',
       'answer_markdown\\2004-数学（文）8.md',
       'answer_markdown\\2004-数学（文）老课程.md',
       'answer_markdown\\2004-数学（理）.md',
       'answer_markdown\\2004-数学（理）7.md',
       'answer_markdown\\2004-数学（理）78.md',
       'answer_markdown\\2004年上海市普通高等学校春季招生考.md',
       'answer_markdown\\2004数学辽宁.md', 'answer_markdown\\2004数学（文）3.md',
       'answer_markdown\\2004数学（理）天津.md', 'answer_markdown\\2004春季题.md',
       'answer_markdown\\2005年高考文科数学全国卷Ⅱ试题及答案（黑龙江、吉林、广西、内蒙古、新疆等地区用）.md',
       'answer_markdown\\2005年高考文科数学全国卷Ⅲ试题及答案（川、陕、云、甘）.md',
       'answer_markdown\\2005年高考文科数学（山东卷）试题及答案.md',
       'answer_markdown\\2005年高考文科数学（江西卷）试题及答案.md',
       'answer_markdown\\2005年高考文科数学（湖南卷）试题及答案.md',
       'answer_markdown\\2005年高考理科数学全国卷Ⅱ试题及答案（黑、吉、广西、内、新）.md',
       'answer_markdown\\2005年高考理科数学全国卷Ⅲ试题及答案（四川、陕西、云南、甘肃等地区用）.md',
    

In [54]:
for _, row in df_include_answer.iloc[10000:12050].iterrows():
    print(row['question'])
    print("************************************************************")
    print(row['answer'])
    print("\n=======================================================================\n")

NameError: name 'df_include_answer' is not defined

In [39]:
df[df['answer'].notnull()]

,file_path,question,answer,is_full,is_error
0,answer_markdown\13（精校版）2019年上海卷数学高考试题文档版（含答案）.md,1\. 已知集合，，则 [ ]{.underline},1\.,True,False
1,answer_markdown\13（精校版）2019年上海卷数学高考试题文档版（含答案）.md,2\. 已知，且满足，求 [ ]{.underline},2\. ，,True,False
2,answer_markdown\13（精校版）2019年上海卷数学高考试题文档版（含答案）.md,3\. 已知向量，，则与的夹角为 [ ]{.underline},3\. ，,True,False
3,answer_markdown\13（精校版）2019年上海卷数学高考试题文档版（含答案）.md,4\. 已知二项式，则展开式中含项的系数为 [ ]{.underline},4\. ，的系数为,True,False
4,answer_markdown\13（精校版）2019年上海卷数学高考试题文档版（含答案）.md,5\. 已知、满足，求的最小值为 [ ]{.underline},5\. ，线性规划作图，后求出边界点代入求最值，当，时，,True,False
...,...,...,...,...,...
25878,answer_markdown\（精校版）2020年浙江高考数学7月试题文档版（含答案）.md,22．（本题满分15分）\n已知，函数，其中e=2.71828...是自然对数的底数．\n（...,22．本题主要考查函数的单调性、零点，导数的运算及其应用，同时考查数学抽象、逻辑推理与数学运...,True,False
25891,answer_markdown\（精校版）2020年浙江高考通用信息技术7月选考试题文档版（...,13.小李收集了手机网民互联网应用的用户规模数据，并用Eel软件进行数据处理，如图a所示。\...,13.（1）=（D4-\$C4）/\$C4\100 （2）C8：F8 （3）①③或③①,True,False
25892,answer_markdown\（精校版）2020年浙江高考通用信息技术7月选考试题文档版（...,"14.小李使用Flash软件创作""花中君子""多媒体作品。请回答下列问题：\n（1）作品的模块...","14.（1）BC （2）矢量图 不会 （3）ABE\n（4）在""文字""图层第31帧处执行""插...",True,False
25893,answer_markdown\（精校版）2020年浙江高考通用信息技术7月选考试题文档版（...,15.某校为学生期末考试分配考场，并编制准考证号。每个班级有班号，每位学生有班内序号，班内序...,15.（1）A （2）①f=True ②m ③num（1）=0或等价表达式\n（3）room...,True,False
